In [1]:
import numpy as np

In [2]:
#possible spectral emissions given as (n, n_prime) which is (sender, reciever)
spectral_transitions = [(7, 6), (7, 5), (7, 4), (7, 3), (7, 2), (7, 1),
                              (6, 5), (6, 4), (6, 3), (6, 2), (6, 1),
                                      (5, 4), (5, 3), (5, 2), (5, 1),
                                              (4, 3), (4, 2), (4, 1),
                                                      (3, 2), (3, 1), 
                                                              (2, 1)]

#possible spectral width modulations
spectral_widths = ["normal", "doubly-ext", "UV-ext", "IR-ext", "UV-dim", "IR-dim"]

#possible spectral filtering
spectral_filters = ["none", "tri", "tri-inv", "lo", "hi", "bp"]


#possible sonic transitions given as [(n, n_prime), can_be_free]
sonic_transitions = [[(6, 2), False],
                     [(5, 2), True],
                     [(4, 2), False],
                     [(7, 3), False], 
                     [(6, 3), True],
                     [(7, 4), True],
                     [(7, 5), False], 
                     [(6, 4), False],
                     [(5, 3), False]]

#possible sonic amplitude modulations 
sonic_amps = ["up", "down", "spike"]

#possible gender transitions
gender_transitions = [(7, 1), (6, 1), (5, 1), (4, 1), (3, 1), (2, 1)]
possible_genders = [2, 3, 4, 5, 6, 7]

In [3]:
class Spectral:
    trans = None
    width = None
    filter = None
    
    def __init__(self, trans, width, filter):
        self.trans = trans
        self.width = width
        self.filter = filter 
    
    def __repr__(self):
        return str(self.trans) + " " + self.width + " " + self.filter
        
    def __str__(self):
        return str(self.trans) + " " + self.width + " " + self.filter
        
        
class Sonic:
    trans = None
    amp = None
    
    def __init__(self, trans, amp):
        self.trans = trans
        self.amp = amp
        
    def __repr__(self):
        return str(self.trans) + " " + self.amp
        
    def __str__(self):
        return str(self.trans) + " " + self.amp
        
        
class Morpheme:
    spectral = None
    sonic1 = None
    sonic2 = None
    sonic3 = None  #should be sonic 
    gender = None  #should be list []
    haptic = None
    
    def __init__(self, spectral, sonic1, gender, haptic, sonic2 = None, sonic3 = None):
        self.spectral = spectral 
        self.sonic1 = sonic1
        self.gender = gender 
        self.haptic = haptic
        if sonic2 is not None:
            self.sonic2 = sonic2
        if sonic3 is not None:
            self.sonic3 = sonic3
            
    def __repr__(self):
        string = ""
        string += 'spectral: ' + str(self.spectral) + "\n"
        string += 'sonic1: ' + str(self.sonic1) + "\n"
        
        if self.sonic2 is not None:
            string += 'sonic2: ' + str(self.sonic2) + "\n"
        if self.sonic3 is not None:
            string += 'sonic3: ' + str(self.sonic3) + "\n"
        string += 'gender: ' + str(self.gender) + "\n"
        string += 'haptic: ' + str(self.haptic) + "\n"
        return string


In [4]:
def generate_random_spectral():
    #get random transition
    #get random width
    #get random filter
    trans = spectral_transitions[np.random.randint(len(spectral_transitions))]
    width = np.random.choice(spectral_widths)
    filter = np.random.choice(spectral_filters)
    
    spectral = Spectral(trans, width, filter)
    return spectral

def generate_random_sonic():
    #get random transition
    #if it can be free, add free to the list
    #get random amp
    
    trans_bool = sonic_transitions[np.random.randint(len(sonic_transitions))]
    trans = trans_bool[0]
    free = trans_bool[1]
    
    possible_amps = sonic_amps.copy()
    if free:
        possible_amps.append("free")

    amp = np.random.choice(possible_amps)
    
    sonic = Sonic(trans, amp)
    return sonic 

def generate_random_gender(): #same as generating random haptic 
    gender = []
    
    num_genders = np.random.randint(7)
    gender_pool = possible_genders.copy()
    
    for i in range(num_genders):
        gender_chosen = np.random.choice(gender_pool)
        gender.append(gender_chosen)
        gender_pool.remove(gender_chosen)
        
    return gender



def generate_random_morpheme():
    #make random spectral
    #make 1-3 random sonic
    #make random gender 
    #put it into morpheme 
    spectral = generate_random_spectral()
    sonic1 = generate_random_sonic()
    sonic2 = generate_random_sonic()
    while sonic2.trans == sonic1.trans:
        sonic2 = generate_random_sonic()
    sonic3 = generate_random_sonic() 
    while sonic3.trans == sonic2.trans or sonic3.trans == sonic1.trans:
        sonic3 = generate_random_sonic()
    
    num_sonic = np.random.randint(3, size=1)[0] + 1
    
    if num_sonic == 2:
        sonic3 = None
    elif num_sonic == 1:
        sonic3 = None
        sonic2 = None
        
    gender = generate_random_gender()
    haptic = generate_random_gender()
        
    morpheme = Morpheme(spectral, sonic1, gender, haptic, sonic2 = sonic2, sonic3 = sonic3)
    return morpheme 

def generate_random_word(): 
    #make 6 morphemes 
    #put into a list 
    word = []
    for i in range(6):
        morph = generate_random_morpheme()
        word.append(morph)
    
    return word

def generate_random_sentence():
    #make 21 words
    #put into a list
    sentence = []
    for i in range(21):
        word = generate_random_word()
        sentence.append(word)
        
    return sentence

In [5]:
#hand movement mapping from (width, filter) --> (home-spec for n, home-spec for n_prime)
hand_movt_mapping = {
    ('normal', 'none'): ("I<N", "I<N"),
 ('doubly-ext', 'none'): ("I<E", "I<E"),
 ('UV-ext', 'none'): ("I<E", "I<N"),
 ('IR-ext', 'none'): ("I<N", "I<E"),
 ('UV-dim', 'none'): ("I<D", "I<N"),
 ('IR-dim', 'none'): ("I<N", "I<D"),
    
 ('normal', 'tri'): ("D<N", "D<N"), 
 ('doubly-ext', 'tri'): ("D<E", "D<E"),
 ('UV-ext', 'tri'): ("D<E", "D<N"),
 ('IR-ext', 'tri'): ("D<N", "D<E"),
 ('UV-dim', 'tri'): ("D<D", "D<N"),
 ('IR-dim', 'tri'): ("D<N", "D<D"),
    
 ('normal', 'tri-inv'): ("U<N", "U<N"),
 ('doubly-ext', 'tri-inv'): ("U<E", "U<E"),
 ('UV-ext', 'tri-inv'): ("U<E", "U<N"),
 ('IR-ext', 'tri-inv'): ("U<N", "U<E"),
 ('UV-dim', 'tri-inv'): ("U<D", "U<N"),
 ('IR-dim', 'tri-inv'): ("U<N", "U<D"),
    
 ('normal', 'lo'): ("U<N", "D<N"),
 ('doubly-ext', 'lo'): ("U<E", "D<E"),
 ('UV-ext', 'lo'): ("U<E", "D<N"),
 ('IR-ext', 'lo'): ("U<N", "D<E"),
 ('UV-dim', 'lo'): ("U<D", "D<N"),
 ('IR-dim', 'lo'): ("U<N", "D<D"),
    
 ('normal', 'hi'): ("D<N", "U<N"),
 ('doubly-ext', 'hi'): ("D<E", "U<E"),
 ('UV-ext', 'hi'): ("D<E", "U<N"),
 ('IR-ext', 'hi'): ("D<N", "U<E"),
 ('UV-dim', 'hi'): ("D<D", "U<N"),
 ('IR-dim', 'hi'): ("D<N", "U<D"),
    
 ('normal', 'bp'): ("O<N", "O<N"),
 ('doubly-ext', 'bp'): ("O<E", "O<E"),
 ('UV-ext', 'bp'): ("O<E", "O<N"),
 ('IR-ext', 'bp'): ("O<N", "O<E"),
 ('UV-dim', 'bp'): ("O<D", "O<N"),
 ('IR-dim', 'bp'): ("O<N", "O<D")
}


In [6]:
#turn word/sentence into video sequencing of actions 


#lists will be [(action_description, length)] in order 
#action onset is at beginning of timelength and lasts for entire timelength 
#except for hand movements, which will only play for one beat and then return to home position (as filmed)
class Video:
    node_2 = None
    node_3 = None
    node_4 = None
    node_5 = None
    node_6 = None
    node_7 = None
    nodes = None
    
    morph_length = 6
    
    def __init__(self, encoding):
        self.node_2 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.node_3 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.node_4 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.node_5 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.node_6 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.node_7 = {"hand":[], "voice":[], "slap": [], "eyes": []}
        self.nodes = {
            2: self.node_2, 
            3: self.node_3, 
            4: self.node_4, 
            5: self.node_5, 
            6: self.node_6, 
            7: self.node_7}
        if len(encoding) == 6:
            self.load_word(encoding)
        elif len(encoding) == 21:
            self.load_sentence(encoding)
            
    def load_word(self, word):
        #print("loading word", word)
        for morpheme in word:
            self.load_morpheme(morpheme)
            
        #fix hands ending thing at the end of words, here
        self.fix_hands()
        
    def fix_hands(self):
        for node in self.nodes:
            hand_list = self.nodes[node]['hand']
            #print(hand_list)
            
            if hand_list[-1][0] is None:
                #check for all None
                if all(x[0] is None for x in hand_list):
                    home_movt = "I<"
                    for i in range(len(hand_list)):
                        hand_list[i] = (home_movt, self.morph_length / 2)
                        
                    print(hand_list)
                    
                else:
                    i = -1 
                    while hand_list[i][0] is None:
                        i -= 1
                    home_movt = hand_list[i][0][0] + "<"
                    i+= 1
                    while i <= -1:
                        hand_list[i] = (home_movt, self.morph_length / 2) 
                        i+= 1
                
        
    def load_morpheme(self, morph):
        #print("loading morpheme", morph)
        
        #do spectral first 
        self.load_spectral(morph.spectral)
        
        #then do sonic
        if morph.sonic3 is not None:
            self.load_sonic(morph.sonic1, 3)
            self.load_sonic(morph.sonic2, 3)
            self.load_sonic(morph.sonic3, 3)
        elif morph.sonic2 is not None:
            self.load_sonic(morph.sonic1, 2)
            self.load_sonic(morph.sonic2, 2)
        else:
            self.load_sonic(morph.sonic1, 1)
            
        #then load gender
        self.load_gender(morph.gender)
        
        #then load haptic
        self.load_haptic(morph.haptic)
        
    def load_sonic(self, sonic, num_sonic):
        #add whistle notes to hi/lo nodes 
        trans = sonic.trans
        hi_node = trans[0]
        lo_node = trans[1]
        
        hi_node_list = self.nodes[hi_node]["voice"]
        lo_node_list = self.nodes[lo_node]["voice"]
        
        hi_node_list.append((str(trans), self.morph_length / num_sonic))
        lo_node_list.append((str(trans), self.morph_length / num_sonic))
        
        #add humming notes to all other nodes (or HAH if free)
        amp = sonic.amp
        nodes = [2, 3, 4, 5, 6, 7]
        nodes.remove(hi_node)
        nodes.remove(lo_node)
        
        if amp == "up":
            for node in nodes[0:3]: #lo note gets more representation (start from lo --> hi nodes)
                node_list = self.nodes[node]["voice"]
                node_list.append((str(lo_node), self.morph_length / num_sonic))
            #then add hi note to remaining node
            for node in nodes[-1:]:
                node_list = self.nodes[node]["voice"]
                node_list.append((str(hi_node), self.morph_length / num_sonic))
                
        elif amp == "down":
            for node in nodes[0:1]: #lo note less representation (start from lo --> hi nodes)
                node_list = self.nodes[node]["voice"]
                node_list.append((str(lo_node), self.morph_length / num_sonic))
            #then add hi note to other three nodes
            for node in nodes[1:]:
                node_list = self.nodes[node]["voice"]
                node_list.append((str(hi_node), self.morph_length / num_sonic))
                
        elif amp == "spike":
            for node in nodes[0:2]: #equal rep 
                node_list = self.nodes[node]["voice"]
                node_list.append((str(lo_node), self.morph_length / num_sonic))
            #then add hi note to other two
            for node in nodes[2:]:
                node_list = self.nodes[node]["voice"]
                node_list.append((str(hi_node), self.morph_length / num_sonic))
                
        elif amp == "free": #add HAH to rest 
            for node in self.nodes:
                if node != hi_node and node != lo_node:
                    node_list = self.nodes[node]["voice"]
                    node_list.append(("HAH", self.morph_length / num_sonic))
        
        return
    
    def load_gender(self, gender):
        #affects head turning / eyes opening 
        #if gender is present, open eyes & head forward
        #AXE THE HEAD TURNING!!
        for gen in gender:
            self.nodes[gen]["eyes"].append(("open", self.morph_length))
        
        for node in self.nodes:
            if node not in gender:
                self.nodes[node]["eyes"].append(("closed", self.morph_length))
        return
    
    def load_haptic(self, haptic):
        #to schlap or not to schlap...that is the question
        for hap in haptic:
            self.nodes[hap]["slap"].append(("slap", self.morph_length))
        
        for node in self.nodes:
            if node not in haptic:
                self.nodes[node]['slap'].append(("none", self.morph_length))
        return
    
    def load_spectral(self, spectral):     
        #get spectral info 
        spectral_trans = spectral.trans 
        spectral_hi_node = spectral_trans[0]
        spectral_lo_node = spectral_trans[1]
        
        #get hand movement sequences 
        width_filt = (spectral.width, spectral.filter)
        hand_movts = hand_movt_mapping[width_filt]
        
        hi_movt = (hand_movts[0], self.morph_length/2)
        hi_home = (hand_movts[0][0] + "<", self.morph_length/2)
        
        lo_home = (hand_movts[1][0] + "<", self.morph_length/2)
        lo_movt = (hand_movts[1], self.morph_length/2)
        
        #for hi node:
        hi_node_list = self.nodes[spectral_hi_node]["hand"]
        #if necessary, change prev. hi_node Nones to hi_home nodes
        if hi_node_list[-1:] != [] and hi_node_list[-1][0] == None:
            i = len(hi_node_list) - 1
            while i >= 0 and hi_node_list[i][0] == None:
                hi_node_list[i] = hi_home
                i -= 1
        #then add movement and then home
        hi_node_list.append(hi_movt)
        hi_node_list.append(hi_home)
    
        
        #for lo node:
        #IF NOT ONE: 
        if spectral_lo_node != 1:
            lo_node_list = self.nodes[spectral_lo_node]["hand"]
            #change prev. lo node Nones to lo_home nodes 
            if lo_node_list[-1:] != [] and lo_node_list[-1][0] == None:
                i = len(lo_node_list) - 1
                while i >= 0 and lo_node_list[i][0] == None:
                    lo_node_list[i] = lo_home
                    i -= 1
            #add home and then movement 
            lo_node_list.append(lo_home)
            lo_node_list.append(lo_movt)
            
            #and then add placeholder to all other nodes 
            for node in self.nodes.keys():
                if node != spectral_hi_node and node != spectral_lo_node:
                    self.nodes[node]["hand"].append((None, self.morph_length/2))
                    self.nodes[node]["hand"].append((None, self.morph_length/2))
        
        else: #if lo node is 1, then add home/movt to ALL OTHER NODES (and change prev. Nones)
            for node in self.nodes.keys():
                if node != spectral_hi_node:
                    node_list = self.nodes[node]["hand"]
                    #change prev Nones
                    if node_list[-1:] != [] and node_list[-1][0] == None:
                        i = len(node_list) - 1
                        while i >= 0 and node_list[i][0] == None:
                            node_list[i] = lo_home
                            i -= 1
                    node_list.append(lo_home)
                    node_list.append(lo_movt)
        
        
    def load_sentence(self, sentence):
        for word in sentence:
            self.load_word(word)
        
        
    def __repr__(self):
        string = ""
        for node, sequence in self.nodes.items():
            string += str(node) + "\n"
            string += "hand: " + str(sequence["hand"]) + "\n"
            string += "voice: " + str(sequence["voice"]) + "\n"
            string += "eyes: " + str(sequence["eyes"]) + "\n"
            string += "slap: " + str(sequence["slap"]) + "\n"
            
        return string
    
    def __str__(self):
        return self.__repr__()
        

In [7]:
#turn action sequence into clip sequencing
def get_clip_sequencing(video):
    node_2 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    node_3 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    node_4 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    node_5 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    node_6 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    node_7 = {"hand":[], "voice":[], "slap": [], "eyes": []}
    clip_sequencing = {
        2: node_2, 
        3: node_3, 
        4: node_4, 
        5: node_5, 
        6: node_6, 
        7: node_7}
    
    #don't need to do anything for sonic
    for node in clip_sequencing:
        clip_sequencing[node]["voice"] = video.nodes[node]["voice"]
    
    #for slap, just add a none at the end
    for node in clip_sequencing:
        clip_sequencing[node]["slap"] = video.nodes[node]["slap"]
        clip_sequencing[node]["slap"].append(("none", 6))
        
    #for eyes
    for node in clip_sequencing:
        eyes_action_sequence = video.nodes[node]["eyes"]
        for i in range(len(eyes_action_sequence)):
            action = eyes_action_sequence[i]
            
            #print(action)
            
            if i == 0:
                clip_sequencing[node]["eyes"].append((action[0][0], 6))
                
            else:
                prev_action = clip_sequencing[node]["eyes"][-1]
                #print(prev_action)
                if prev_action[0][-1] == action[0][0]:
                    clip_sequencing[node]["eyes"].append((action[0][0], 6))
                else:
                    clip_sequencing[node]["eyes"].append((prev_action[0][-1] + action[0][0], 6))
                    
        #then add the last bit
        prev = clip_sequencing[node]['eyes'][-1][0][-1]
        clip_sequencing[node]['eyes'].append((prev, 6))
                
            
        
    
    #do spectral 
    for node in clip_sequencing:
        spectral_action_sequence = video.nodes[node]["hand"]
        for i in range(len(spectral_action_sequence)):
            action = spectral_action_sequence[i]
            #print(action)
            #first one is a little different...
            if i == 0:
                #just add it - HOLD or movt will be ok 
                clip_sequencing[node]["hand"].append(action)
            else:
                prev_action = spectral_action_sequence[i-1]
                curr_home = action[0][0]
                prev_home = prev_action[0][0]
                
                if prev_home != curr_home:
                    if len(prev_action[0]) == 2 and len(action[0]) == 2:
                        clip_sequencing[node]["hand"].append((prev_action[0] + action[0], 3))
                        #print("home to home", prev_action[0], action[0])
                        
                    elif len(prev_action[0]) == 2 and len(action[0]) == 3:
                        clip_sequencing[node]["hand"][-1] = (prev_action[0], 1.0)
                        clip_sequencing[node]["hand"].append((prev_home + "<" + curr_home + "<", 2))
                        clip_sequencing[node]["hand"].append((action[0], 3))
                       # print(clip_sequencing[node]["hand"][-3:])
                        
                    elif len(prev_action[0]) == 3 and len(action[0]) == 2:
                        clip_sequencing[node]["hand"].append((prev_home + "<" + action[0], 3))
                        #print("movt to home", prev_action[0], action[0])
                        
                    elif len(prev_action[0]) == 3 and len(action[0]) == 3:
                        clip_sequencing[node]["hand"][-1] = (prev_action[0], 2.5)
                        clip_sequencing[node]["hand"].append((prev_home + "<" + curr_home + "<", 1))
                        clip_sequencing[node]["hand"].append((action[0], 2.5))
                        #clip_sequencing[node]["hand"].append(("MOVE TO MOVE THING"))
                        
                       # print("movt to movt", prev_action[0], action[0])
                       # print(clip_sequencing[node]["hand"][-3:])
                        
                    
                        #print("?>??????????")
                        #print(prev_action[0], action[0], i)
                #print(prev_home, curr_home)
                
                else:
                    clip_sequencing[node]["hand"].append(action)
                    
        #then add the last bit, to end with 2sec extra
        prev_home = clip_sequencing[node]["hand"][-1][0][0]
        clip_sequencing[node]["hand"].append((prev_home + "<", 3.0))
            
        #print()
    return clip_sequencing

In [8]:
sentence =  generate_random_sentence()

In [16]:
sentence[0][0]

spectral: (6, 1) UV-ext tri
sonic1: (5, 2) down
sonic2: (6, 3) spike
sonic3: (6, 2) up
gender: [np.int64(7), np.int64(2), np.int64(6), np.int64(4)]
haptic: [np.int64(5), np.int64(3), np.int64(4), np.int64(2)]

In [18]:
import json
with open('../HE-DEMS sentence encodings (scores)/random_sentence_1.json') as f:
    rand_1 = json.load(f)

In [21]:
sentence_vid = Video(sentence)

In [33]:
len(sentence_vid.node_2['slap'])

127

In [34]:
sentence_vid.node_2['slap'][0]

('slap', 6)

In [27]:
sentence_clip_sequence = get_clip_sequencing(sentence_vid)

In [30]:
len(sentence_clip_sequence[2]['hand'])

257

In [ ]:
#first, generate random word or sentence
word = generate_random_word()
sentence = generate_random_sentence()

#then, get video action sequence from those
word_vid = Video(word)
sentence_vid = Video(sentence)

#then, get clip sequencing from those

word_clip_sequencing = get_clip_sequencing(word_vid)
sentence_clip_sequencing = get_clip_sequencing(sentence_vid)

#then, save sentence sequencing to json
import json
with open("sentence-encoding.json", "w") as file:
    json.dump(sentence_clip_sequencing, file)

In [8]:
#load file from json
import json
with open("embodied performance/RANDOM-SPEECH/sentence-encoding.json", "r") as file:
    sentence_encoding = json.load(file)

In [26]:
from moviepy.editor import VideoFileClip, concatenate_videoclips, concatenate_audioclips, AudioFileClip
import os

#create video clips from sequencing info
node_2_folderpath = "embodied performance/1. marquise/EDITED/"
node_3_folderpath = "embodied performance/2. putri/EDITED/"
node_4_folderpath = "embodied performance/3. rachel/EDITED/"
node_5_folderpath = "embodied performance/4. lacie/EDITED/"
node_6_folderpath = "embodied performance/5. reshma/EDITED/"
node_7_folderpath = "embodied performance/6. nancy/EDITED/"
node_folders = {
    2: node_2_folderpath,
    3: node_3_folderpath,
    4: node_4_folderpath,
    5: node_5_folderpath,
    6: node_6_folderpath,
    7: node_7_folderpath
}

SHORT_start = "00:00:00.08"
SHORT_end = "00:00:01.04"

start_2 = 0
end_2 = 1.5

start_25 = 0
end_25 = 1.875

def make_video_clips(sequencing, name):
    #first, make folder to store everything
    #print("HELOWERIJ WEO")
    store_folder = "embodied performance/RANDOM-SPEECH/" + name + "/"
    if not os.path.exists(store_folder):
        os.mkdir(store_folder)
    
    #go node by node 
    for node in sequencing:
        #print(node)
        info = sequencing[str(node)]
        #print(info)
        hand_info = info["hand"]
        eyes_info = info['eyes']
        slap_info = info['slap']
        voice_info = info['voice']
        
        #print(slap_info)
        
        #only do marquise for now lol 
        if int(node) == 2:
        #do hands
            make_hands_video(node, name, store_folder, hand_info)
        #then eyes
            #make_eyes_video(node, name, store_folder, eyes_info)
        #then slap
            #make_slap_video(int(node), name, store_folder, slap_info)
        #then voice 
            #make_voice_video(node, name, store_folder, voice_info)
        
def make_hands_video(node, name, store_folder, hand_info):
    
    #where to look for clips
    source_folder = node_folders[int(node)] + "HANDS/"
    print(source_folder)
    #where to put final concatenated video
    hands_clip_filename = store_folder + str(node) + "-hands"
    #where to store clips in order
    single_clip_list = []
    concatenated_clip_list = []
    
    total_length = 0
    i = 0
    
    #iterate through info
    for info in hand_info:
        #get filename and length
        clip_filename = info[0].replace("<", "-") + ".mp4"
        print(info[0], clip_filename)
        clip_filename = source_folder + clip_filename
        clip_length = info[1]
        
        #make clip based on length
        if clip_length == 3.0 or clip_length == 3: 
            clip = VideoFileClip(clip_filename)
        elif clip_length == 2.5:
            clip = VideoFileClip(clip_filename).subclip(start_25, end_25)
        elif clip_length == 2.0 or clip_length == 2:
            clip = VideoFileClip(clip_filename).subclip(start_2, end_2)
        elif clip_length == 1.0 or clip_length == 1:
            clip = VideoFileClip(clip_filename).subclip(SHORT_start, SHORT_end)
            
        else:
            print("heello????)")
           
        #save clip to list 
        single_clip_list.append(clip)
        total_length += clip_length
        
        print(total_length)
        
        if total_length == 36 or total_length == 36.0 or total_length == 72.0 or total_length == 144.0:
            print("consolidating morpheme")
            if total_length == 36.0:
                concatenated_clip = concatenate_videoclips(single_clip_list)
                concatenated_clip_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                concatenated_clip.write_videofile(concatenated_clip_filename)
                print(concatenated_clip_filename)
                concatenated_clip_list.append(concatenated_clip_filename)
                i += 1
                
            elif total_length == 72.0:
                concatenated_clip = concatenate_videoclips(single_clip_list)
                concat_1 = concatenated_clip.subclip(0, 27)
                concat_1_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_2 = concatenated_clip.subclip(27, 54)
                concat_2_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_1.write_videofile(concat_1_filename)
                concatenated_clip_list.append(concat_1_filename)
                concat_2.write_videofile(concat_2_filename)
                concatenated_clip_list.append(concat_2_filename)
                
                
            elif total_length == 144.0:
                concatenated_clip = concatenate_videoclips(single_clip_list)
                concat_1 = concatenated_clip.subclip(0, 27)
                concat_1_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_2 = concatenated_clip.subclip(27, 54)
                concat_2_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_3 = concatenated_clip.subclip(54, 81)
                concat_3_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_4 = concatenated_clip.subclip(81, 108)
                concat_4_filename = hands_clip_filename + "-" + str(i) + ".mp4"
                i += 1
                concat_1.write_videofile(concat_1_filename)
                concatenated_clip_list.append(concat_1_filename)
                concat_2.write_videofile(concat_2_filename)
                concatenated_clip_list.append(concat_2_filename)
                concat_3.write_videofile(concat_3_filename)
                concatenated_clip_list.append(concat_3_filename)
                concat_4.write_videofile(concat_4_filename)
                concatenated_clip_list.append(concat_4_filename)
                
                
                
            for clip in single_clip_list:
                clip.close()
                
            single_clip_list = []
            total_length = 0
            
    
    #should be one single clip left .... 
    print(len(single_clip_list))
    print(len(concatenated_clip_list))
    
    all_clips = [VideoFileClip(clip).subclip(0, 27) for clip in concatenated_clip_list]
    all_clips += single_clip_list
    
    hands_clip = concatenate_videoclips(all_clips)
    hands_clip.write_videofile(hands_clip_filename + ".mp4")
    
    
    return
    
               
def make_eyes_video(node, name, store_folder, eyes_info):
    
    #where to look for clips
    source_folder = node_folders[int(node)] + "EYES/"
    print(source_folder)
    #where to put final concatenated video
    eyes_clip_filename = store_folder + str(node) + "-eyes"
    #where to store clips in order
    single_clip_list = []
    concatenated_clip_list = []
    
    total_length = 0
    i = 0
    
    for info in eyes_info:
        clip_filename = info[0] + ".mp4"
        print(info[0], clip_filename)
        clip_filename = source_folder + clip_filename
        clip = VideoFileClip(clip_filename)
        single_clip_list.append(clip)
        total_length += 6
        
        if total_length == 36:
            print("consolidating morpheme")
            concatenated_clip = concatenate_videoclips(single_clip_list)
            concatenated_clip_filename = eyes_clip_filename + "-" + str(i) + ".mp4"
            concatenated_clip.write_videofile(concatenated_clip_filename)
            print(concatenated_clip_filename)
            concatenated_clip_list.append(concatenated_clip_filename)
                
            for clip in single_clip_list:
                clip.close()
                
            single_clip_list = []
            total_length = 0
            i += 1
        
    #should be one single clip left .... 
    print(len(single_clip_list))
    print(len(concatenated_clip_list))
    
    all_clips = [VideoFileClip(clip).subclip(0, 27) for clip in concatenated_clip_list]
    all_clips += single_clip_list
    
    eyes_clip = concatenate_videoclips(all_clips)
    eyes_clip.write_videofile(eyes_clip_filename + ".mp4")
        
    return
    
def make_slap_video(node, name, store_folder, slap_info):
    
    """
    MAKE SLAP TRACK AS WELL!!!!!!
    """
    
    #where to look for clips
    source_folder = node_folders[int(node)] + "SLAP/"
    print(source_folder)
    #where to put final concatenated video
    slap_clip_filename = store_folder + str(node) + "-slap"
    slap_audio_filename = slap_clip_filename + "-audio"
    #where to store clips in order
    clip_list = []
    single_clip_list = []
    concatenated_clip_list = []
    
    single_audio_list = []
    concat_audio_list = []
    slap_source_folder = "embodied performance/SLAP/"
    total_length = 0
    i = 0
    
    
    for info in slap_info:
        clip_filename = info[0] + ".mp4"
        print(info[0], clip_filename)
        clip_filename = source_folder + clip_filename
        clip = VideoFileClip(clip_filename)
        single_clip_list.append(clip)
        
        if info[0] == "slap":
            audio_filename = slap_source_folder + info[0] + "-0" + str(np.random.randint(8)+1) + ".wav"
            print(os.path.exists(audio_filename))
        else:
            audio_filename = slap_source_folder + "none.wav"
            
        audio = AudioFileClip(audio_filename)
        single_audio_list.append(audio)
            
        total_length += 6
        
        if total_length == 36:
            print("consolidating morpheme")
            concatenated_clip = concatenate_videoclips(single_clip_list)
            concatenated_clip_filename = slap_clip_filename + "-" + str(i) + ".mp4"
            concatenated_clip.write_videofile(concatenated_clip_filename)
            print(concatenated_clip_filename)
            concatenated_clip_list.append(concatenated_clip_filename)
                
            for clip in single_clip_list:
                clip.close()
                
            single_clip_list = []
            
            concat_audio = concatenate_audioclips(single_audio_list)
            concat_audio_filename = slap_audio_filename + "-" + str(i) + ".wav"
            concat_audio.write_audiofile(concat_audio_filename)
            print(concat_audio_filename)
            concat_audio_list.append(concat_audio_filename)
            
            for clip in single_audio_list:
                clip.close()
                
            single_audio_list = []
            
            total_length = 0
            i += 1
        
    #should be one single clip left .... 
    print(len(single_clip_list))
    print(len(concatenated_clip_list))
    
    print(len(single_audio_list))
    print(len(concat_audio_list))
    
    all_clips = [VideoFileClip(clip).subclip(0, 27) for clip in concatenated_clip_list]
    all_clips += single_clip_list
    
    slap_clip = concatenate_videoclips(all_clips)
    slap_clip.write_videofile(slap_clip_filename + ".mp4")
    
    all_audio = [AudioFileClip(clip).subclip(0, 27) for clip in concat_audio_list]
    all_audio += single_audio_list
    
    slap_audio = concatenate_audioclips(all_audio)
    slap_audio.write_audiofile(slap_audio_filename + ".wav")
    
    return
    
def make_voice_video(node, name, store_folder, voice_info):
    
    #where to look for clips
    source_folder = node_folders[int(node)] + "VOICE/"
    print(source_folder)
    #where to put final concatenated video
    voice_clip_filename = store_folder + str(node) + "-voice"
    #where to store clips in order
    single_clip_list = []
    concatenated_clip_list = []
    total_length = 0
    i = 0
    
    for info in voice_info:
        noise = info[0]
        length = info[1] 
        
        if len(noise) == 1: #hum 
            clip_filename = noise
            if length == 6.0:
                clip_filename += "-01.wav"
            elif length == 3.0:
                clip_filename += "-02.wav"
            elif length == 2.0:
                clip_filename += "-03.wav"
                
        elif len(noise) == 3: #HAH
            clip_filename = noise
            if length == 6.0: 
                clip_filename += "-6"
            elif length == 3.0:
                clip_filename += "-3"
            elif length == 2.0:
                clip_filename += "-2"
            clip_filename += "-0" + str(np.random.randint(3)+1)+".wav"
            
        elif len(noise) == 6:
            clip_filename = noise.replace(",", "")
            if length == 6.0:
                clip_filename += "-01.wav"
            elif length == 3.0:
                clip_filename += "-02.wav"
            elif length == 2.0:
                clip_filename += "-03.wav"
        print(noise, length, clip_filename)
        
        clip_filename = source_folder + clip_filename
        clip = AudioFileClip(clip_filename)
        single_clip_list.append(clip)
        
        total_length += length
        
        print(total_length)
        
        
        if total_length == 36.0:
            print("consolidating morpheme")
            concatenated_clip = concatenate_audioclips(single_clip_list)
            concatenated_clip_filename = voice_clip_filename + "-" + str(i) + ".wav"
            concatenated_clip.write_audiofile(concatenated_clip_filename)
            print(concatenated_clip_filename)
            concatenated_clip_list.append(concatenated_clip_filename)
                
            for clip in single_clip_list:
                clip.close()
                
            single_clip_list = []
            total_length = 0
            i += 1
        
    print(len(single_clip_list))
    print(len(concatenated_clip_list))
    
    all_clips = [AudioFileClip(clip).subclip(0, 27) for clip in concatenated_clip_list]
    
    voice_clip = concatenate_audioclips(all_clips)
    voice_clip.write_audiofile(voice_clip_filename + ".mp3")
    
    return


In [27]:
make_video_clips(sentence_encoding, "SENTENCE/2")

embodied performance/1. marquise/EDITED/HANDS/
U< U-.mp4
3.0
U<D U-D.mp4
6.0
U<O< U-O-.mp4
9.0
O< O-.mp4
12.0
O< O-.mp4
15.0
O< O-.mp4
18.0
O< O-.mp4
21.0
O<E O-E.mp4
24.0
O<U< O-U-.mp4
27.0
U<N U-N.mp4
30.0
U< U-.mp4
33.0
U< U-.mp4
34.0
U<O< U-O-.mp4
36.0
consolidating morpheme
Moviepy - Building video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-0.mp4.
MoviePy - Writing audio in 2-hands-0TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-0.mp4



Moviepy - Done !
Moviepy - video ready embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-0.mp4
embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-0.mp4
O<N O-N.mp4
3
O< O-.mp4
6.0
O< O-.mp4
9.0
O<E O-E.mp4
12.0
O<U< O-U-.mp4
15.0
U< U-.mp4
18.0
U< U-.mp4
21.0
U< U-.mp4
24.0
U< U-.mp4
27.0
U<E U-E.mp4
30.0
U< U-.mp4
33.0
U<E U-E.mp4
36.0
consolidating morpheme
Moviepy - Building video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-1.mp4.
MoviePy - Writing audio in 2-hands-1TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-1.mp4



Moviepy - Done !
Moviepy - video ready embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-1.mp4
embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-1.mp4
U<D< U-D-.mp4
3
D< D-.mp4
6.0
D< D-.mp4
9.0
D< D-.mp4
12.0
D< D-.mp4
15.0
D< D-.mp4
18.0
D< D-.mp4
21.0
D<D D-D.mp4
24.0
D<U< D-U-.mp4
27.0
U< U-.mp4
30.0
U< U-.mp4
33.0
U<D U-D.mp4
36.0
consolidating morpheme
Moviepy - Building video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-2.mp4.
MoviePy - Writing audio in 2-hands-2TEMP_MPY_wvf_snd.mp3


...

Moviepy - Done !
Moviepy - video ready embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-19.mp4
embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-19.mp4
I<D< I-D-.mp4
3
D<N D-N.mp4
6.0
D< D-.mp4
9.0
D<N D-N.mp4
12.0
D<U< D-U-.mp4
15.0
U<E U-E.mp4
18.0
U< U-.mp4
21.0
U<E U-E.mp4
24.0
U< U-.mp4
27.0
U< U-.mp4
30.0
U< U-.mp4
33.0
U< U-.mp4
36.0
consolidating morpheme
Moviepy - Building video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-20.mp4.
MoviePy - Writing audio in 2-hands-20TEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-20.mp4



Moviepy - Done !
Moviepy - video ready embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-20.mp4
embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands-20.mp4
U< U-.mp4
3.0
1
21
Moviepy - Building video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands.mp4.
MoviePy - Writing audio in 2-handsTEMP_MPY_wvf_snd.mp3


MoviePy - Done.
Moviepy - Writing video embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands.mp4



Moviepy - Done !
Moviepy - video ready embodied performance/RANDOM-SPEECH/SENTENCE/2/2-hands.mp4


In [ ]:
#generate randomized word-sentences from existing words 
sentence_folder = "embodied performance/RANDOM-SPEECH/SENTENCE/"

def randomize_sentence(sentence_affix):
    
    #output concat files will be "2-eyes-sentenceaffix.mp4" etc
    
    #first, grab all filenames 
    all_node_files = {
    }
    
    for i in range(2, 8):
        
        node_hands = [sentence_folder + str(i) + "/" + str(i) + "-hands-" + str(j) + ".mp4" for j in range(21)]
        node_eyes = [sentence_folder + str(i) + "/" + str(i) + "-eyes-" + str(j) + ".mp4" for j in range(21)]
        node_voice = [sentence_folder + str(i) + "/" + str(i) + "-voice-" + str(j) + ".wav" for j in range(21)]
        node_slap = [sentence_folder + str(i) + "/" + str(i) + "-slap-" + str(j) + ".mp4" for j in range(21)]
        node_slap_audio = [sentence_folder + str(i) + "/" + str(i) + "-slap-audio-" + str(j) + ".wav" 
                           for j in range(21)]
    
        node_files = {
            "hands": node_hands,
            "eyes": node_eyes,
            "voice": node_voice,
            "slap": node_slap,
            "slap-audio": node_slap_audio
        }
    
        all_node_files[i] = node_files
    
    #then, get a random word order
    order = [i for i in range(21)]
    np.random.shuffle(order)
    
    print(order)
    
    #for each node:
    for node in all_node_files:
        node_info = all_node_files[node]
        #for each type of action
        for action in node_info:
            info = node_info[action]
            #concatenate using order & export video / audio 
            if action == "hands" or action == "eyes" or action == "slap":
                #order video clips 
                clips_ordered = [VideoFileClip(info[i]).subclip(0, 27) for i in order]
                
                #get concat name
                concat_clip_name = sentence_folder + str(node) + "/" + sentence_affix + "/" + sentence_affix 
                concat_clip_name += "-" + str(node) + "-" + action + ".mp4"
                
                #concatenate & export 
                concat_clip = concatenate_videofiles(clips_ordered)
                concat_clip.write_videofile(concat_clip_name)
                
            else:
                #order audio clips 
                clips_ordered = [AudioFileClip(info[i]).subclip(0, 27) for i in order]
                
                #get concat name 
                concat_clip_name = sentence_folder + str(node) + "/" + sentence_affix + "/" + sentence_affix 
                concat_clip_name += "-" + str(node) + "-" + action + ".wav"
                
                #concatenate & export 
                concat_clip = concatenate_audiofiles(clips_ordered)
                concat_clip.write_audiofile(concat_clip_name)

                
    
    
    

In [ ]:
randomize_sentence("speaker2")
randomize_sentence("speaker3")
randomize_sentence("speaker4")
randomize_sentence("speaker5")
randomize_sentence("speaker6")

In [ ]:
sentence_encoding["3"]['hand'][100:150]

In [23]:
path = "embodied performance/RANDOM-SPEECH/SENTENCE/6/6-voice"

clips = []
for i in range(21):
    clip_path = path + "-" + str(i) + ".wav"
    if i != 7:
        clips.append(AudioFileClip(clip_path).subclip(0, 27))
    else:
        clips.append(AudioFileClip(clip_path))
        

full_clip = concatenate_audioclips(clips)
full_clip.write_audiofile(path + ".wav")

MoviePy - Writing audio in embodied performance/RANDOM-SPEECH/SENTENCE/6/6-voice.wav


MoviePy - Done.
